In [ ]:
#最初にマウント→メールアドレス選択→許可→「Mounted at /content/drive」を確認
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 訓練データの読み込み
import csv
import numpy  as np
import pandas as pd
import glob
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import lightgbm as lgb

path = '/content/drive/My Drive/2023_Nishika_aki_train'
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename)
    li.append(df)

df_train_org = pd.concat(li, axis=0, ignore_index=True)


# 特徴量（絞り込み済）＋目的変数
df_train_X_y = df_train_org.iloc[:,[3,7,8,9,10,14,24,27]]


# all出力
# df_train_X_y.to_csv('/content/drive/My Drive/2023_Nishika_aki_train/all/all_train_X_y_1.csv')


# ＮＧ値をＯＫ値へ置き換え
df_train_X_y = df_train_X_y.replace('1H?1H30', '75')
df_train_X_y = df_train_X_y.replace('1H30?2H', '105')
df_train_X_y = df_train_X_y.replace('2H?', '120')
df_train_X_y = df_train_X_y.replace('30分?60分', '45')
df_train_X_y = df_train_X_y.replace('2000㎡以上', '2000')


df_train_X_y.columns = ['post','eki','minute','mad','area','born','bai','price']

# ＮａＮを空白へ置き換え
df_train_X_y['eki'] = df_train_X_y['eki'].fillna('空白')
df_train_X_y['mad'] = df_train_X_y['mad'].fillna('空白')
df_train_X_y['born'] = df_train_X_y['born'].fillna('空白')

# ＮａＮを平均値へ置き換え
#やむを得ず
df_train_X_y['minute'] = df_train_X_y['minute'].fillna(11.61216821)

#各特徴量ごとの目的変数の平均をとる
post_means = df_train_X_y.groupby('post')['price'].mean()
eki_means  = df_train_X_y.groupby('eki')['price'].mean()
mad_means  = df_train_X_y.groupby('mad')['price'].mean()
born_means = df_train_X_y.groupby('born')['price'].mean()
bai_means  = df_train_X_y.groupby('bai')['price'].mean()


# 目的変数の平均を特徴量として列を置き換える（ターゲットエンコーディング）
df_train_X_y['post'] = df_train_X_y['post'].map(post_means)
df_train_X_y['eki']  = df_train_X_y['eki'].map(eki_means)
df_train_X_y['mad']  = df_train_X_y['mad'].map(mad_means)
df_train_X_y['born'] = df_train_X_y['born'].map(born_means)
df_train_X_y['bai']  = df_train_X_y['bai'].map(bai_means)

df_train_X_y.iloc[:,2]  = df_train_X_y.iloc[:,2].astype('float')
df_train_X_y.iloc[:,4]  = df_train_X_y.iloc[:,4].astype('float')


# all出力
# df_train_X_y.to_csv('/content/drive/My Drive/2023_Nishika_aki_train/all/all_train_X_y_3.csv')

# 改めて学習データをＸ（特徴量）とｙ（目的変数）へ分割
df_train_X = df_train_X_y.iloc[:,:7]
df_train_y = df_train_X_y.iloc[:,7]


# 学習データを、訓練データと検証データへ分割する


X_train, X_valid, y_train, y_valid = train_test_split(
    df_train_X, df_train_y, test_size=0.1, random_state=0
    )


# 評価指標として決定係数（MAE）を使う



# 機械学習アルゴリズムにｌｉｇｈｔＧＢＭを使用



trains = lgb.Dataset(X_train,y_train)
valids = lgb.Dataset(X_valid,y_valid)


params = {
  'objective':'regression',
  'metrics':'mae',
}


model = lgb.train(params, trains, valid_sets=valids, num_boost_round=100000,
                  callbacks=[lgb.early_stopping(stopping_rounds=100)])


# 予測
y_train_pred = model.predict(X_train)
y_valid_pred = model.predict(X_valid)

# pickleをインポート
import pickle

# モデルを保存
# filename = 'ml_model.sav'
filename = '/content/drive/My Drive/2023_Nishika_aki_train/all/ml_model.sav'
pickle.dump(model, open(filename, 'wb'))



<ipython-input-3-a4fcb652e8ad>:14: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)
<ipython-input-3-a4fcb652e8ad>:14: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)
<ipython-input-3-a4fcb652e8ad>:14: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)
<ipython-input-3-a4fcb652e8ad>:14: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)
<ipython-input-3-a4fcb652e8ad>:62: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_train_X_y.iloc[:,2]  = df_tra

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.052456 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 781
[LightGBM] [Info] Number of data points in the train set: 714039, number of used features: 7
[LightGBM] [Info] Start training from score 7.236887
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[11152]	valid_0's l1: 0.0849826


In [8]:
def price_get(ent_minute, ent_area):
    filename = 'ml_model.sav'
    model_load = pickle.load(open(filename, 'rb'))

    df_real_X = pd.DataFrame(columns=['post', 'eki', 'minute', 'mad', 'area', 'born', 'bai'])

    # 特徴量
    #
    df_real_X = df_real_X.append({'post': 6.994685678,  #市区町村コード
                                  'eki': 6.938155853,   #最寄り駅(博多)
                                  'minute': ent_minute, #最寄り駅からの時間(分)
                                  'mad': 7.046666262,   #間取り(1K)
                                  'area': ent_area,     #面積
                                  'born': 6.943949703,  #建築年(平成3年)
                                  'bai': 7.172629839},  #取引時点(2012年)
                    ignore_index=True)

    # 予測
    y_real_pred = model_load.predict(df_real_X)

    # 'return'を使って、関数の戻り値を指定します。
    return  y_real_pred

price_log = price_get(10, 70)[0]
price = round(10**price_log/10000)*10000
print(price)


8980000


<ipython-input-8-f5053b3bd053>:9: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_real_X = df_real_X.append({'post': 6.994685678,  #市区町村コード
